In [16]:
# Define your data directories
train_dir = 'COP_banknote/ds3/Train'
test_dir = 'COP_banknote/ds3/Test'
validation_dir = 'COP_banknote/ds3/Validation'

In [31]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Load the VGG16 model without the top layer (include_top=False)
base_model =  VGG16(weights='imagenet', include_top=False)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(12, activation='softmax')(x)  # Use 'softmax' for multi-class classification

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers from the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Create data generators
batch_size = 32
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # InceptionV3 input size
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    classes=['100k', '100kf', '10k', '10kf', '50k', '50kf', '5k', '5kf', '20k', '20kf', '2k', '2kf']
)

# Validation data generator
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    classes=['100k', '100kf', '10k', '10kf', '50k', '50kf', '5k', '5kf', '20k', '20kf', '2k', '2kf']
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=3,  # You can adjust the number of epochs
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Evaluate on test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    classes=['100k', '100kf', '10k', '10kf', '50k', '50kf', '5k', '5kf', '20k', '20kf', '2k', '2kf']
)

test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f'InceptionV3 Test accuracy: {test_acc * 100:.2f}%')

# Save the trained model for later use
model.save('banknote_inceptionv3_multiclass.h5')


Found 8600 images belonging to 12 classes.
Found 2913 images belonging to 12 classes.
Epoch 1/3
269/269 [==============================] - 267s 990ms/step - loss: 1.4485 - accuracy: 0.4777 - val_loss: 0.8520 - val_accuracy: 0.7638
Epoch 2/3
269/269 [==============================] - 242s 899ms/step - loss: 0.7864 - accuracy: 0.7441 - val_loss: 0.4997 - val_accuracy: 0.8541
Epoch 3/3
269/269 [==============================] - 214s 796ms/step - loss: 0.5346 - accuracy: 0.8292 - val_loss: 0.4962 - val_accuracy: 0.8177
Found 2903 images belonging to 12 classes.
91/91 [==============================] - 52s 572ms/step - loss: 0.4901 - accuracy: 0.8185
InceptionV3 Test accuracy: 81.85%
